## Checking a degree-2 irreducible representation for $S_4$

In [1]:
# Initialization

import numpy as np
import numpy.linalg as npla

S4 = SymmetricGroup(4)
print(f"S4: {S4}")
print(f" some of the {len(S4)} elements of S4:", S4[:20:3])
print()

# Identity matrix for future use

I = np.array([[1,0], [0,1]])
print("Identity matrix I:\n", I)
print()

# Get a primitive cube root of unity. 
# Probably should do this symbolically somehow in Sage, but for now just use floats.

q = (-1 + 1j*np.sqrt(3)) / 2
qq = q*q

print("Cube roots of unity:")
print(f" q:   {q}")
print(f" q^2: {qq}")
print(f" q^3:  {qq*q}")
print()

# A crude test for close-enough-to-0 in well-conditioned floating-point

def is_tiny(x):
    return npla.norm(x) < 10**-8

S4: Symmetric group of order 4! as a permutation group
 some of the 24 elements of S4: [(), (1,2)(3,4), (1,4,3), (1,3,4), (3,4), (1,2), (1,4)]

Identity matrix I:
 [[1 0]
 [0 1]]

Cube roots of unity:
 q:   -0.500000000000000 + 0.866025403784439*I
 q^2: -0.500000000000000 - 0.866025403784439*I
 q^3:  1.00000000000000 + 1.11022302462516E-16*I



In [2]:
# As an aside, look at one of the matrices for a 3-cycle

M = np.array([[qq,0], [0,q]])

print("M:\n", M)
print()
print("trace(M):", np.trace(M), "  det(M):", npla.det(M))
print()
print("M^3:\n", M@M@M)
print()
print(" is_tiny(M^3 - I):", is_tiny(M@M@M-I))
print()

M:
 [[-0.5-0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]

trace(M): (-0.9999999999999999+0j)   det(M): (1+1.5909636962210474e-16j)

M^3:
 [[1.+2.22044605e-16j 0.+0.00000000e+00j]
 [0.+0.00000000e+00j 1.+9.61481343e-17j]]

 is_tiny(M^3 - I): True



In [3]:
# Start with three matrices for transpositions, all with trace 0 and det -1
# We'll use one for each pair of nonoverlapping transpositions.

Ta = np.array([[0,1], [1,0]])
Tb = np.array([[0,q], [qq,0]])
Tc = np.array([[0,qq],[q,0]])

print("Ta:\n", Ta, "\n\nTb:\n", Tb, "\n\nTc:\n", Tc)
print()

print("Are the traces all 0?", is_tiny(np.trace(Ta)), is_tiny(np.trace(Tb)), is_tiny(np.trace(Tc)))
print("Are the squares all the identity?", is_tiny(Ta@Ta-I), is_tiny(Tb@Tb-I), is_tiny(Tc@Tc-I))
print()
print("Tb @ Tc:\n", Tb@Tc)
print()
print("Is Tb@Tc the same as M?", is_tiny(Tb@Tc-M))
print()

Ta:
 [[0 1]
 [1 0]] 

Tb:
 [[ 0. +0.j        -0.5+0.8660254j]
 [-0.5-0.8660254j  0. +0.j       ]] 

Tc:
 [[ 0. +0.j        -0.5-0.8660254j]
 [-0.5+0.8660254j  0. +0.j       ]]

Are the traces all 0? True True True
Are the squares all the identity? True True True

Tb @ Tc:
 [[-0.5-0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]

Is Tb@Tc the same as M? True



In [4]:
# Use the matrices representing the transpositions to set up a dictionary
# of the matrices representing all the elements of S4. 

transpositions = { S4((1,2)): Ta, S4((3,4)): Ta,
                   S4((1,3)): Tb, S4((2,4)): Tb,
                   S4((1,4)): Tc, S4((2,3)): Tc }
rep = transpositions.copy()

# Every remaining element of S4 is a product of 2 or 3 transpositions.
# This is truly ugly brute force, but should be fast enough.

for g1 in transpositions:
    for g2 in transpositions:
        g = g1 * g2
        if g not in rep:
            rep[g] = rep[g1] @ rep[g2]
        for g3 in transpositions:
            g = g1 * g2 * g3
            if g not in rep:
                rep[g] = rep[g1] @ rep[g2] @ rep[g3]
            
print("Did we find all the elements of S4?", len(rep) == len(S4))
print()

Did we find all the elements of S4? True



In [5]:
# Look at the representations

for g in S4:
    print(f"{g}:\n", rep[g])
    print("trace: ", int(np.trace(rep[g]).real.round()))
    print()

():
 [[1 0]
 [0 1]]
trace:  2

(1,3)(2,4):
 [[1.+1.11022302e-16j 0.+0.00000000e+00j]
 [0.+0.00000000e+00j 1.+9.61481343e-17j]]
trace:  2

(1,4)(2,3):
 [[1.+9.61481343e-17j 0.+0.00000000e+00j]
 [0.+0.00000000e+00j 1.+1.11022302e-16j]]
trace:  2

(1,2)(3,4):
 [[1 0]
 [0 1]]
trace:  2

(2,3,4):
 [[-0.5+0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5-0.8660254j]]
trace:  -1

(1,3,2):
 [[-0.5+0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5-0.8660254j]]
trace:  -1

(1,4,3):
 [[-0.5+0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5-0.8660254j]]
trace:  -1

(1,2,4):
 [[-0.5+0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5-0.8660254j]]
trace:  -1

(2,4,3):
 [[-0.5-0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]
trace:  -1

(1,3,4):
 [[-0.5-0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]
trace:  -1

(1,4,2):
 [[-0.5-0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0.5+0.8660254j]]
trace:  -1

(1,2,3):
 [[-0.5-0.8660254j  0. +0.j       ]
 [ 0. +0.j        -0

In [6]:
# Go through the whole multiplication table to verify it's a homomorphism

for g in S4:
    for h in S4:
        assert is_tiny(rep[g] @ rep[h] - rep[g * h])
        
print("Yup, rep is a homomorphism!")

Yup, rep is a homomorphism!
